# Exercise 1: SEIRS Model for a Different Country

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/epistorm/tech-transfer-epydemix/blob/main/sessions/session-4/exercises/python/exercise_1_seirs_model.ipynb)

**Objective:** Build an SEIRS model (with waning immunity) and simulate it for a country of your choice.

**Skills practiced:**
- Defining compartments and transitions
- Loading population data
- Running and visualizing simulations

In [ ]:
# Colab installation (skip if running locally)
import sys, os, subprocess
if "google.colab" in sys.modules or os.getenv("COLAB_RELEASE_TAG"):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r",
                    "https://raw.githubusercontent.com/epistorm/epydemix/refs/heads/main/tutorials/colab_requirements.txt"])

## Task 1: Create the SEIRS Model

The SEIRS model extends SEIR by adding waning immunity (R → S transition).

**Compartments:** S, E, I, R

**Transitions:**
- S → E: infection (mediated by contact with I)
- E → I: progression to infectious (spontaneous)
- I → R: recovery (spontaneous)
- R → S: waning immunity (spontaneous) — the key addition

**Parameters:**
- β = 0.03 (transmission rate)
- σ = 0.2 (1/5 day latent period)
- γ = 0.1 (1/10 day infectious period)
- ω = 0.01 (1/100 day immunity duration)

In [ ]:
from epydemix import EpiModel

# TODO: Create SEIRS model with compartments S, E, I, R
model = ...

# TODO: Add transitions
# S → E: infection (mediated by I)

# E → I: progression to infectious

# I → R: recovery

# R → S: waning immunity (the key addition for SEIRS)

# TODO: Set parameters (beta, sigma, gamma, omega)

print(model)

## Task 2: Load Population Data

Choose a country from the [supported locations](https://github.com/epistorm/epydemix-data/blob/main/locations.csv) and load its population data.

In [ ]:
from epydemix.population import load_epydemix_population

# TODO: Load population data for a country of your choice
population = ...

# TODO: Attach population to the model

print(population)

### Visualize Population and Contacts

In [ ]:
from epydemix.visualization import plot_population, plot_contact_matrix
import matplotlib.pyplot as plt

# TODO: Visualize population distribution and contact matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=150)

# Plot population

# Plot contact matrix (e.g., "community" layer)

plt.tight_layout()

## Task 3: Run Simulations and Visualize

Run 50 stochastic simulations over 1 year, seeding with approximately 10 infected individuals.

In [ ]:
import numpy as np

# TODO: Run 50 simulations over 1 year
# Hint: percentage_in_agents = 10 / population.Nk.sum() seeds ~10 infected
results = ...

### Visualize All Compartments Over Time

In [ ]:
from epydemix.visualization import plot_quantiles

# TODO: Get quantiles and plot all compartments (S, E, I, R)
df_quantiles = ...

fig, ax = plt.subplots(figsize=(12, 5), dpi=150)
# Plot S_total, E_total, I_total, R_total

### Infections by Age Group

In [ ]:
# TODO: Plot infections by age group
# Hint: columns like "I_0-4", "I_5-19", "I_20-49", "I_50-64", "I_65+"
fig, ax = plt.subplots(figsize=(12, 5), dpi=150)

## Task 4: Compare SEIRS vs SEIR

To understand the effect of waning immunity, create a standard SEIR model (without the R → S transition) and compare the dynamics.

In [ ]:
# TODO: Create standard SEIR model (no waning immunity)
# Same as SEIRS but without the R → S transition
model_seir = ...

# TODO: Run simulations with same parameters
results_seir = ...

### Compare Infected and Susceptible Dynamics

In [ ]:
import seaborn as sns
colors = sns.color_palette("Dark2")

# TODO: Compare SEIRS vs SEIR dynamics
# Plot infected over time for both models
# Plot susceptible over time for both models

fig, axes = plt.subplots(1, 2, figsize=(14, 4), dpi=150)

# Infected comparison (axes[0])

# Susceptible comparison (axes[1])

plt.tight_layout()

## Discussion

**How does waning immunity affect the long-term dynamics compared to a standard SEIR model?**

*Write your observations here:*

1. What happens to the susceptible pool over time in each model?

2. Does the SEIRS model show any signs of recurrent waves or endemic behavior?

3. What diseases might be better modeled with SEIRS vs SEIR?